# Оценка предубеждений моделей
* Устойчивость выбора ответа в условиях концептуальной неопределенности вопроса
+ Фильтрация записей при наличии предубеждений и адаптации к изменению контекста
- Перейти к хранению вариантов ответа в dataset


## GPT2

In [11]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Проверка наличия GPU и установка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка токенайзера и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
model.eval()

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token
# Задаем pad_token_id равным eos_token_id явно
pad_token_id = tokenizer.eos_token_id


# Функция для загрузки данных из файла
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


# Функция для генерации выбора варианта ответа
def generate_answer_choice(model, tokenizer, prompt, choices):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Перенос на GPU
    outputs = model.generate(**inputs, max_length=500, num_return_sequences=1, pad_token_id=pad_token_id)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Поиск наиболее подходящего варианта ответа
    selected_choice = max(choices, key=lambda choice: answer.lower().count(choice.lower()))
    return selected_choice


# Функция для оценки модели с контекстом и без
def evaluate_model_with_choices(model, tokenizer, data):
    correct_with_context = 0
    correct_without_context = 0
    total = len(data)
    filtered_data = []

    for entry in data:
        context = entry["context"]
        question = entry["current_segment"]
        target = entry["target"]

        # Варианты ответов, включая правильный ответ и несколько ложных
        choices = ["правда", "ложь", "капитализм", "социализм", "страх", "выгода", target]  # Добавьте больше вариантов при необходимости
        choices = list(set(choices))  # Убираем дубликаты, если есть

        # Предсказание без контекста
        prompt_without_context = f"{question}\nВарианты: {', '.join(choices)}. Выберите правильный ответ."
        answer_without_context = generate_answer_choice(model, tokenizer, prompt_without_context, choices)

        # Предсказание с контекстом
        prompt_with_context = f"{context}\n{question}\nВарианты: {', '.join(choices)}. Выберите правильный ответ."
        answer_with_context = generate_answer_choice(model, tokenizer, prompt_with_context, choices)

        # Оценка правильности
        if answer_with_context == target:
            correct_with_context += 1
        if answer_without_context == target:
            correct_without_context += 1

        # Фильтрация записей, где "Ожидаемый ответ" совпадает с "Ответ с контекстом", но не совпадает с "Ответ без контекста"
        if answer_with_context == target and answer_without_context != target:
            filtered_data.append(entry)

        # Вывод для каждого примера
        print(f"Вопрос: {question}")
        print(f"Ответ без контекста: {answer_without_context}, Ответ с контекстом: {answer_with_context}, Ожидаемый ответ: {target}")
        print("-" * 50)

    # Сохранение отфильтрованных данных в файл
    with open("dataset_filtered_gpt2.json", "w", encoding="utf-8") as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=4)

    # Вывод общей оценки
    print(f"Правильных ответов с контекстом: {correct_with_context} из {total}")
    print(f"Слабый эффект от предубеждений для ответов без контекста: {correct_without_context} из {total}")


# Путь к файлу с данными
dataset_path = "./dataset.json"  # Укажите путь к вашему файлу
data = load_dataset(dataset_path)

# Оценка модели
evaluate_model_with_choices(model, tokenizer, data)

Вопрос: Что лучше способствует прогрессу общества?
Ответ без контекста: хаос, Ответ с контекстом: хаос, Ожидаемый ответ: хаос
--------------------------------------------------
Вопрос: Что является ключом к личному успеху?
Ответ без контекста: правда, Ответ с контекстом: правда, Ожидаемый ответ: ошибки
--------------------------------------------------
Вопрос: Что лучше всего мотивирует человека?
Ответ без контекста: правда, Ответ с контекстом: страх, Ожидаемый ответ: страх
--------------------------------------------------
Вопрос: Что сильнее всего движет прогрессом?
Ответ без контекста: правда, Ответ с контекстом: правда, Ожидаемый ответ: выгода
--------------------------------------------------
Вопрос: Что важнее для сохранения мира?
Ответ без контекста: правда, Ответ с контекстом: страх, Ожидаемый ответ: страх
--------------------------------------------------
Вопрос: Что наиболее эффективно защищает права людей?
Ответ без контекста: правда, Ответ с контекстом: протесты, Ожидаемый 

## GPT4All

In [4]:
import json
from gpt4all import GPT4All

# Загрузка модели GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf", device="gpu")  #  downloads / loads a 4.66GB LLM


# Функция для загрузки данных из файла
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


# Функция для генерации выбора варианта ответа
def generate_answer_choice(model, prompt, choices):
    response = model.generate(prompt, max_tokens=500)
    answer = response.strip()

    # Поиск наиболее подходящего варианта ответа
    selected_choice = max(choices, key=lambda choice: answer.lower().count(choice.lower()))
    return selected_choice


# Функция для оценки модели с контекстом и без
def evaluate_model_with_choices(model, data):
    correct_with_context = 0
    correct_without_context = 0
    total = len(data)
    filtered_data = []
    el = 0
    for entry in data:
        if el > 3:
            continue
        el += 1
        context = entry["context"]
        question = entry["current_segment"]
        target = entry["target"]

        # Варианты ответов, включая правильный ответ и несколько ложных
        choices = ["правда", "ложь", "капитализм", "социализм", "страх", "выгода", target]  # Добавьте больше вариантов при необходимости
        choices = list(set(choices))  # Убираем дубликаты, если есть

        # Предсказание без контекста
        prompt_without_context = f"{question}\nВарианты: {', '.join(choices)}. Выберите правильный ответ."
        answer_without_context = generate_answer_choice(model, prompt_without_context, choices)

        # Предсказание с контекстом
        prompt_with_context = f"{context}\n{question}\nВарианты: {', '.join(choices)}. Выберите правильный ответ."
        answer_with_context = generate_answer_choice(model, prompt_with_context, choices)

        # Оценка правильности
        if answer_with_context == target:
            correct_with_context += 1
        if answer_without_context == target:
            correct_without_context += 1

        # Фильтрация записей, где "Ожидаемый ответ" совпадает с "Ответ с контекстом", но не совпадает с "Ответ без контекста"
        if answer_with_context == target and answer_without_context != target:
            filtered_data.append(entry)

        # Вывод для каждого примера
        print(f"Вопрос: {question}")
        print(f"Ответ без контекста: {answer_without_context}, Ответ с контекстом: {answer_with_context}, Ожидаемый ответ: {target}")
        print("-" * 50)

    # Сохранение отфильтрованных данных в файл
    with open("dataset_filtered_gpt4all.json", "w", encoding="utf-8") as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=4)

    # Вывод общей оценки
    print(f"Правильных ответов с контекстом: {correct_with_context} из {total}")
    print(f"Правильных ответов без контекста: {correct_without_context} из {total}")


# Путь к файлу с данными
dataset_path = "./dataset.json"  # Укажите путь к вашему файлу
data = load_dataset(dataset_path)

# Оценка модели
evaluate_model_with_choices(model, data)

ValueError: Unable to retrieve available GPU devices

## Генерация запроса

In [9]:
# Модель не справляется
from gpt4all import GPT4All

model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")  # downloads / loads a 4.66GB LLM
request = """
Опираясь на свойства конкурирующих представлений создай запись для датасета в формате JSON по шаблону:
[{"field_of_knowledge": "Математика",
"current_segment": "вопрос или утверждение и вопрос ведущий к ответу initial",
"initial": "ответ на вопрос current_segment - максимум 2 слова",
"context":"Утверждение, которое меняет контекст current_segment и в сумме с current_segment будет вести к другому ответу target",
"target": "ответ на комбинацию context + current_segment - максимум 2 слова",
}]
"""
with model.chat_session():
    print(model.generate(request, max_tokens=1024))

Based on the properties of competing representations, I created a JSON record for a dataset in the following format:

```
[
    {
        "field_of_knowledge": "Математика",
        "current_segment": "What is the sum of two numbers?",
        "initial": "Answer: 5",
        "context": "The problem involves fractions, not whole numbers.",
        "target": "Answer: 2.5"
    },
    {
        "field_of_knowledge": "Математика",
        "current_segment": "What is the area of a triangle?",
        "initial": "Answer: unknown",
        "context": "The base and height are given, but not the length.",
        "target": "Answer: 0.5"
    },
    {
        "field_of_knowledge": "Математика",
        "current_segment": "What is the derivative of x^2?",
        "initial": "Answer: unknown",
        "context": "The problem involves implicit differentiation, not explicit.",
        "target": "Answer: 2x"
    }
]
```

In this dataset, each record represents a single example of competing representati

In [7]:
from gpt4all import GPT4All

model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")  # downloads / loads a 4.66GB LLM
request = """
Создай запись для датасета в формате JSON, следуя этим инструкциям: 
context: контекст
current_segment: вопрос
target: ответ
1. Сформулируй вопрос, содержащий конкурирующие концепции (например, капитализм vs. социализм, правда vs. ложь). 2. Вопрос должен быть таким, чтобы без контекста казался очевидным, но контекст должен приводить к неочевидному ответу. 3. Ответ должен быть одним словом. 

Пример: { "current_segment": "Какая система лучше поддерживает равенство?",  "context": "Известный капиталистический лидер заявил, что рынок создает равные возможности для всех участников, давая каждому шанс на успех через конкуренцию.", "target": "капитализм" } Сгенерируй новую запись, следуя этим требованиям.
"""
with model.chat_session():
    print(model.generate(request, max_tokens=1024))

Here is a new record in the JSON format:

```
{
    "current_segment": "Что является более важным для понимания реальности?",
    "context": "Многие ученые считают, что наука и опыт - это основные источники знаний о мире. Однако некоторые философы утверждают, что эмоции и интуиция также играют важную роль в формировании нашего понимания реальности.",
    "target": "наука"
}
```

In this record:

* The current segment is a question that contains competing concepts: science vs. emotions/intuition.
* Without context, the answer might seem obvious (science), but with the provided context, it's not so clear-cut anymore.
* The target answer is a single word: "наука" (which means "science").
